In [40]:
import json
import pickle
import random
from collections import defaultdict
from datetime import datetime
from itertools import chain
from math import log
from typing import Iterable

import keras
import numpy
import pandas
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from matplotlib import pyplot
from scipy import stats
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold
from statsmodels.formula import api as formula_api

%matplotlib inline
pandas.set_option('display.max_columns', 500)

In [41]:
# Define cross-validation and evaluation strategies.

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = 'accuracy'

In [42]:
# Parse battle logs.

def parse_heroes(heroes: Iterable[dict], sign: int, result: dict):
    for hero in heroes:
        result[f'level_{hero["id"]}'] += sign * hero['level']
        result[f'color_{hero["id"]}'] += sign * hero['color']
        result[f'star_{hero["id"]}'] += sign * hero['star']


def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    parse_heroes(battle.get('attackers') or battle['player'], +1, result)
    parse_heroes(battle.get('defenders') or battle['enemies'], -1, result)
        
    return {'win': battle['win'], **result}

In [43]:
# Load the logs into a data frame.

lines = list(chain.from_iterable([
    open('battles.jsonl'),
    open('battles-lilia.jsonl'),
]))

numpy.random.seed(42)
battles = pandas.DataFrame([parse_battle(line) for line in lines]).fillna(value=0)
battles.drop_duplicates(inplace=True)  # because the files may contain duplicate battles
battles.head()

,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_23,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_37,color_38,color_39,color_4,color_40,color_41,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_23,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_37,level_38,level_39,level_4,level_40,level_41,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_23,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_37,star_38,star_39,star_4,star_40,star_41,star_5,star_6,star_7,star_8,star_9,win
0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
1,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,False
2,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False


In [44]:
battles.describe()

,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_23,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_37,color_38,color_39,color_4,color_40,color_41,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_23,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_37,level_38,level_39,level_4,level_40,level_41,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_23,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_37,star_38,star_39,star_4,star_40,star_41,star_5,star_6,star_7,star_8,star_9
count,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000,313.000000
mean,-0.565495,3.316294,1.319489,-1.300319,3.594249,-0.309904,-0.731629,0.115016,-0.309904,0.194888,-0.121406,1.811502,-1.306709,-0.153355,-0.166134,-0.054313,1.520767,-0.373802,-0.239617,-0.070288,-0.095847,-0.143770,-0.156550,0.022364,-0.319489,-0.095847,-0.182109,0.076677,-0.150160,-0.022364,0.207668,-0.194888,-2.562300,-0.111821,0.006390,-0.092652,-0.763578,-0.402556,-0.884984,-0.303514,-4.009585,27.808307,10.482428,-8.661342,28.236422,-2.319489,-5.696486,1.035144,-2.517572,2.961661,-0.971246,13.396166,-10.833866,-1.201278,-1.357827,-0.383387,12.178914,-2.980831,-1.849840,-0.549521,-0.722045,-1.067093,-1.239617,0.175719,-2.568690,-0.869010,-1.405751,0.744409,-1.130990,-0.159744,1.741214,-1.587859,-20.073482,-0.945687,0.006390,-0.763578,-5.507987,-2.821086,-7.146965,-1.619808,-0.191693,1.888179,0.597444,-0.527157,1.597444,-0.111821,-0.175719,0.057508,-0.124601,0.102236,-0.035144,0.926518,-0.466454,-0.060703,-0.051118,-0.031949,0.089457,-0.121406,-0.099042,-0.022364,-0.025559,-0.044728,-0.099042,0.006390,-0.124601,-0.051118,-0.067093,0.025559,-0.041534,-0.009585,0.092652,-0.057508,-0.977636,-0.028754,0.012780,-0.044728,-0.230032,0.159744,-0.335463,-0.166134
std,2.654766,3.838773,3.324372,3.720943,3.918018,1.885283,5.090670,0.824037,1.426556,4.022019,1.300131,3.589460,3.106319,1.241173,1.049055,0.588557,4.167224,1.608536,1.336169,1.323425,0.845661,1.138671,1.119937,0.395663,1.482863,0.952602,1.264176,1.949819,1.219424,0.395663,1.584684,1.147896,4.325326,0.907741,0.079808,0.821024,2.542274,4.673054,2.594286,1.283732,21.009992,31.390351,26.557035,30.263273,30.779744,15.301044,41.168827,7.416331,11.445265,32.633824,10.147114,28.751303

In [45]:
# Split the dataframe.

x = battles.drop(['win'], axis=1)
y = battles['win']

In [46]:
# Train, adjust hyper-parameters and evaluate.

numpy.random.seed(42)

classifier = RandomForestClassifier(class_weight='balanced', n_jobs=5, random_state=42)
param_grid = {
    'n_estimators': [39, 40, 41],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None],
    'criterion': ['entropy', 'gini'],
}

grid_search = GridSearchCV(classifier, param_grid, cv=cv, scoring=scoring, n_jobs=1).fit(x, y)
best_estimator = grid_search.best_estimator_

print(f'Best score: {grid_search.best_score_}')
print(f'Best params: {grid_search.best_params_}')
print(f'Classes: {best_estimator.classes_}')

scores = cross_val_score(best_estimator, x, y, scoring=scoring, cv=cv)
print(f'CV score: {scores.mean()} (std: {scores.std()})')

Best score: 0.8115015974440895
Best params: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 40}
Classes: [False  True]
CV score: 0.811520737327189 (std: 0.030750285633395675)


In [47]:
# Re-train the best model on the entire dataset.

best_estimator.fit(x, y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=5,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [48]:
# Save the model.

print(f'''
"""
Arena battle prediction model.
Auto-generated on {datetime.now().replace(microsecond=0)}.
X shape: {x.shape}.
Score: {scores.mean():.4f} (std: {scores.std():.4f}).
"""

import pickle

from {best_estimator.__class__.__module__} import {best_estimator.__class__.__qualname__}

feature_names = {list(x.columns)}
model: {best_estimator.__class__.__qualname__} = pickle.loads({pickle.dumps(best_estimator)})
'''.strip(), file=open('model.py', 'wt'))

In [49]:
# Display the feature importances.

pandas.DataFrame({'Feature': x.columns, 'Importance': best_estimator.feature_importances_}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False) \
    .head(n=10)

,Importance
Feature,
level_2,0.062250
level_7,0.046183
level_10,0.042015
level_15,0.041542
level_25,0.037011
level_18,0.031210
star_9,0.031004
level_12,0.030699
color_7,0.030470


In [51]:
# Display prediction examples.

result = pandas.concat((
    pandas.Series(best_estimator.predict(x), index=battles.index, name='win_predicted'),
    pandas.Series(best_estimator.predict_proba(x)[:, 1], index=battles.index, name='win_probability'),
    battles,
), axis=1)
result['win_probability'] = result['win_probability'].apply('{:.2f}'.format)
result.head(10)

,win_predicted,win_probability,color_1,color_10,color_11,color_12,color_13,color_14,color_15,color_16,color_17,color_18,color_19,color_2,color_20,color_21,color_22,color_23,color_25,color_26,color_27,color_28,color_29,color_3,color_30,color_31,color_32,color_33,color_34,color_35,color_36,color_37,color_38,color_39,color_4,color_40,color_41,color_5,color_6,color_7,color_8,color_9,level_1,level_10,level_11,level_12,level_13,level_14,level_15,level_16,level_17,level_18,level_19,level_2,level_20,level_21,level_22,level_23,level_25,level_26,level_27,level_28,level_29,level_3,level_30,level_31,level_32,level_33,level_34,level_35,level_36,level_37,level_38,level_39,level_4,level_40,level_41,level_5,level_6,level_7,level_8,level_9,star_1,star_10,star_11,star_12,star_13,star_14,star_15,star_16,star_17,star_18,star_19,star_2,star_20,star_21,star_22,star_23,star_25,star_26,star_27,star_28,star_29,star_3,star_30,star_31,star_32,star_33,star_34,star_35,star_36,star_37,star_38,star_39,star_4,star_40,star_41,star_5,star_6,star_7,star_8,star_9,win
0,False,0.15,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,-5.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,35.0,0.0,-42.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,-7.0,-42.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
1,False,0.00,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,-2.0,-6.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,-10.0,0.0,0.0,35.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-13.0,-46.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,False
2,False,0.05,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,-2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,36.0,0.0,0.0,-9.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-42.0,0.0,0.0,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
3,False,0.00,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,36.0,0.0,0.0,35.0,0.0,-10.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,-1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4,False,0.20,0.0,0.0,0.0,-1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,37.0,0.0,0.0,37.0,0.0,-4.0,0.0,0.0,-41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.0,-41.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,False
5,False,0.00,0.0,0.0,0.0,-3.0,0.0,0.0,-1.0,0.0,0.0,4.0,0.0,-1.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,-13.0,0.0,0.0,37.0,0.0,-9.0,-46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0